In [1]:
#!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 MB 22.7 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 470.6 kB/s eta 0:00:00a 0:00:01
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.90-cp310-cp310-linux_x86_64.whl size=3302203 sha256=2d40e111f232c70fad26c51e6781653da814692706c3a01e3f22de26e945fa05
  Stored in directory: /home/cdsw/.cache/pip/wheels/3d/67/02/f950031435db4a5a02e6269f6adb6703bf1631c3616380f3c6
Successfully built llama-cpp-python


In [1]:
from huggingface_hub import snapshot_download

# Download the GGUF model from Hugging Face
model_dir = snapshot_download(repo_id="TheBloke/Karen_TheEditor_V2_STRICT_Mistral_7B-GGUF")
print(f"Model downloaded to: {model_dir}")


/home/cdsw/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 15 files: 100%|███████████████████████| 15/15 [00:00<00:00, 246.46it/s]

Model downloaded to: /home/cdsw/.cache/huggingface/hub/models--TheBloke--Karen_TheEditor_V2_STRICT_Mistral_7B-GGUF/snapshots/6c654aa207a4b673379db7a928b87a01d644676c


In [1]:
#START HERE WHEN MODEL IS ALREADy aVAILABLE
from llama_cpp import Llama
model_path = "/home/cdsw/.cache/huggingface/hub/models--TheBloke--Karen_TheEditor_V2_STRICT_Mistral_7B-GGUF/snapshots/6c654aa207a4b673379db7a928b87a01d644676c/karen_theeditor_v2_strict_mistral_7b.Q8_0.gguf"

In [2]:
!nvidia-smi

Tue Sep 17 08:50:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           On  | 00000000:00:1C.0 Off |                    0 |
| N/A   37C    P0              36W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
# GPU Memory is 16384MiB 

# Initialize the model with GPU support
llm = Llama(
    model_path=model_path,
    n_gpu_layers=-1,  # -1 means use all available GPU layers
    n_ctx=16384,  # adjust based on your GPU memory
)


llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from /home/cdsw/.cache/huggingface/hub/models--TheBloke--Karen_TheEditor_V2_STRICT_Mistral_7B-GGUF/snapshots/6c654aa207a4b673379db7a928b87a01d644676c/karen_theeditor_v2_strict_mistral_7b.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = fpham_karen_theeditor_v2_strict_mistr...
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32    

In [8]:
#Prompt Playground
# let is try using a text
text_to_correct =  'I saw a smaple product. These products is below expected quality. \n I start to understand what he said is quite     right.  \
                    The correct value is 12.5  % and 3   r d position. The time is 10:30 am  \
                   The length of the object is 845mm and cost $1m. The sunrise happens at 0750 EST and visibily is 35 km.  \
                   The right way to summarize and favor some advisors is still to be found'
prompt = (
    "<|system|>You are an expert editor who corrects spelling, formatting and grammatical errors<|end|>\n"
    "<|user|> ** TASK**  \n" 
    f"1. Edit the following text for spelling and grammar mistakes:  '{text_to_correct}'\n"
    f"2. Remove any formatting errors such as extra spaces"
    f"**IMPORTANT** use the following template to format your response **."
    f"1. Edited Text: "
    f"[ Your corrected text here ]"
    f"2. Corrections: "
    f"[ Make a numbered list of your corrections ]"
    "<|end|>\n"
    "<|assistant|>\n"
)
# Generate the output using the model
output = llm(prompt, max_tokens=400, temperature=0.7, top_p=0.1, top_k=40, repeat_penalty=1.18)
#print(output['choices'][0]['text'].strip())
# Print the list of corrections
print(f"ORIGINAL TEXT : #########  \n {text_to_correct} \n\n #########")
print(f"CORRECTED TEXT : #########  \n {output['choices'][0]['text'].strip()}\n \n ################# ")

Llama.generate: 222 prefix-match hit, remaining 20 prompt tokens to eval

llama_print_timings:        load time =    6530.36 ms
llama_print_timings:      sample time =      87.29 ms /   132 runs   (    0.66 ms per token,  1512.13 tokens per second)
llama_print_timings: prompt eval time =     680.69 ms /    20 tokens (   34.03 ms per token,    29.38 tokens per second)
llama_print_timings:        eval time =   16905.61 ms /   131 runs   (  129.05 ms per token,     7.75 tokens per second)
llama_print_timings:       total time =   17784.25 ms /   151 tokens


ORIGINAL TEXT : #########  
 I saw a smaple product. These products is below expected quality. 
 I start to understand what he said is quite     right.                      The correct value is 12.5  % and 3   r d position. The time is 10:30 am                     The length of the object is 845mm and cost $1m. The sunrise happens at 0750 EST and visibily is 35 km.                     The right way to summarize and favor some advisors is still to be found 

 #########
CORRECTED TEXT : #########  
 1. Edited Text: 'I saw a sample product. These products are below expected quality. I start to understand what he said is quite right. The correct value is 12.5% and 3rd position. The time is 10:30 am. The length of the object is 845mm and costs $1m. The sunrise happens at 0750 EST and visibility is 35 km. The right way to summarize and favor some advisors is still to be found.'
2. Corrections: [a. Correction 1 - Remove extra spaces]
 
 ################# 
